In [22]:
import pandas as pd
import os
import glob

def convert_parquet_to_txt(dataset_name, text_key="text"):
    text = ""
    pattern = f"wikitext/wikitext-103-raw-v1/{dataset_name}-*-of-*.parquet"
    parquet_files = glob.glob(pattern)
    
    for parquet_file in parquet_files:
        df = pd.read_parquet(parquet_file)
        text += "".join(df[text_key])
        
    os.makedirs("wikitext/wikitext-103-raw-v1-txt", exist_ok=True)
    output_path = f"wikitext/wikitext-103-raw-v1-txt/{dataset_name}.txt"
    
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(text)

convert_parquet_to_txt("train")
convert_parquet_to_txt("validation") 
convert_parquet_to_txt("test")

In [36]:
from datasets import load_dataset
ds = load_dataset("simplescaling/s1K-1.1")["train"]

def create_text(entry):
    text = \
        entry["question"] + '\n' + \
        entry["gemini_thinking_trajectory"] + \
        entry["gemini_attempt"]
    return {"text": text}

ds = ds.map(create_text)

# Split into train/val/test
ds = ds.train_test_split(test_size=0.2, seed=42)
test_valid = ds["test"].train_test_split(test_size=0.5, seed=42)

train_ds = ds["train"]
valid_ds = test_valid["train"] 
test_ds = test_valid["test"]

# Write to txt files
os.makedirs("s1", exist_ok=True)

with open("s1/train.txt", "w", encoding="utf-8") as f:
    f.write("".join(train_ds["text"]))
    
with open("s1/validation.txt", "w", encoding="utf-8") as f:
    f.write("".join(valid_ds["text"]))
    
with open("s1/test.txt", "w", encoding="utf-8") as f:
    f.write("".join(test_ds["text"]))